In [1]:
# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [3]:
# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28711 images belonging to 7 classes.


In [4]:
# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [6]:
# create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

In [7]:
emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [8]:
# Train the neural network/model
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

Epoch 1/50


C:\Users\izhar\AppData\Local\Temp\ipykernel_13524\3198193097.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


448/448 [==============================] - 208s 460ms/step - loss: 1.8008 - accuracy: 0.2610 - val_loss: 1.7139 - val_accuracy: 0.3511
Epoch 2/50
448/448 [==============================] - 183s 408ms/step - loss: 1.6190 - accuracy: 0.3703 - val_loss: 1.5409 - val_accuracy: 0.4184
Epoch 3/50
448/448 [==============================] - 192s 428ms/step - loss: 1.5264 - accuracy: 0.4123 - val_loss: 1.4626 - val_accuracy: 0.4413
Epoch 4/50
448/448 [==============================] - 969s 2s/step - loss: 1.4605 - accuracy: 0.4428 - val_loss: 1.4104 - val_accuracy: 0.4623
Epoch 5/50
448/448 [==============================] - 181s 403ms/step - loss: 1.3995 - accuracy: 0.4646 - val_loss: 1.3560 - val_accuracy: 0.4823
Epoch 6/50
448/448 [==============================] - 258s 576ms/step - loss: 1.3521 - accuracy: 0.4857 - val_loss: 1.3227 - val_accuracy: 0.4961
Epoch 7/50
448/448 [==============================] - 276s 616ms/step - loss: 1.3008 - accuracy: 0.5063 - val_loss: 1.2798 - val_accuracy:

In [9]:
# save model structure in jason file
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')

In [13]:
# Training Ends here